In [1]:
import sys
sys.path.append("../..")
import numpy as np
import matplotlib.pyplot as plt
from pyfusion import Fusion, AnalogTwo
%matplotlib qt

In [639]:
def load_bitstream_to_flash(cmd_if:serial.Serial, data_if:serial.Serial, path:str, partition_index=0):
    print(f"CMD if = {cmd_if.port}  DATA if = {data_if.port}")
    with open(path, 'rb') as bs:
        data=bs.read()#[26:]
        bs_size=len(data)
        cmd = bytearray(5)
        print(f"Writing bitstream to flash, size={bs_size}")
        cmd[0]=4
        cmd[1]=0
        cmd[2]=0
        cmd[3]=0
        cmd[4]=partition_index&255
        print(cmd)
        cmd_if.write(cmd)
        data_if.write(data)
        remaining = int(1024*1024/8 - bs_size)
        print(f"Stuff with {remaining} x 0xff")
        data_if.write(bytearray([255]* remaining ))

In [2]:
analog_two = AnalogTwo(bitstream_path='../../rtl/top/analog_two/build/top.bit')
analog_two.open()

CMD if = /dev/ttyACM0  DATA if = /dev/ttyACM1
Loading bitstream, size=126318
bytearray(b'\x01\x00\x01\xedn')


In [3]:
v=analog_two.get_samples(count=200000)

In [ ]:
fig,ax= plt.subplots(4, 1)
for a,c in zip(ax,v):
    a.plot(c)

In [4]:
fig,ax= plt.subplots(4, 1)
for a,c in zip(ax,v):
    fft = np.fft.fft(c-np.mean(c))
    freq = np.fft.fftfreq(len(fft), d=1./2e5)[0:len(fft)//2]
    fft = np.abs(fft[0:len(fft)//2])
    a.plot(freq, fft)
    a.semilogx()

In [376]:
load_bitstream(serial.Serial(find_device("CMD")), serial.Serial(find_device("DATA")), '../../rtl/top/test_FMC/build/top.bit')

CMD if = /dev/ttyACM2  DATA if = /dev/ttyACM3
Loading bitstream, size=101727
bytearray(b'\x01\x00\x01\x8d_')


In [622]:
load_bitstream(serial.Serial(find_device("CMD")), serial.Serial(find_device("DATA")), '../../rtl/top/analog_two/build/top.bit')

CMD if = /dev/ttyACM1  DATA if = /dev/ttyACM2
Loading bitstream, size=125975
bytearray(b'\x01\x00\x01\xec\x17')


In [653]:
load_bitstream_to_flash(serial.Serial(find_device("CMD")), serial.Serial(find_device("DATA")), '../../rtl/top/analog_two/build/top.bit')

CMD if = /dev/ttyACM25  DATA if = /dev/ttyACM26
Writing bitstream to flash, size=125719
bytearray(b'\x04\x00\x00\x00\x00')


KeyboardInterrupt: 

In [ ]:
582368

In [111]:
bs=bs.read()

In [114]:
bs[26:128]

b'\x00\xff\xff\xff\xbd\xb3\xff\xff\xff\xff;\x00\x00\x00\xe2\x00\x00\x00!\x11\x10C"\x00\x00\x00@\x00\x00\x00F\x00\x00\x00\x82\x91\x1d\x8a\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

In [627]:
d=serial.Serial(find_device("DATA"))
d.flushInput()
v=d.read(4096*128)

In [365]:
%timeit d.read(20_000_000)

4.9 s ± 2.03 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
np.where(np.array(list(v), dtype=np.uint8)==0xff)

In [142]:
print(v[4080:4110])

b'\xed\x80\x00\x00\xee\x80\x00\x00\xef\x80\x00\x00\xff\x00\xff\x00\x00\xff\x00\xff\xc9f\x00\x00\xcaf\x00\x00\xcbf'


In [497]:
plt.figure()
data = np.diff(np.frombuffer(v, dtype=np.uint32))
data[data>1e7]=0
plt.plot(data)

In [560]:
values = np.frombuffer(v, dtype=np.int16).copy()

In [545]:
for value,svalue in zip(np.frombuffer(v, dtype=np.uint16),np.frombuffer(v, dtype=np.int16)):
    if value==0xf0f0:
        print("================")
    print(f"{np.binary_repr(value, width=16)}\t{svalue}\t{hex(value)}")


1111111111110110	-10	0xfff6
1111111111111010	-6	0xfffa
0000000000000000	0	0x0
0000000000001100	12	0xc
1111000011110000	-3856	0xf0f0
0111011101000000	30528	0x7740
1111111111111110	-2	0xfffe
1111111111111000	-8	0xfff8
1111111111110000	-16	0xfff0
0000000000010000	16	0x10
1111000011110000	-3856	0xf0f0
0111011101000001	30529	0x7741
0000000000000110	6	0x6
1111111111111010	-6	0xfffa
0000000000000000	0	0x0
0000000000010010	18	0x12
1111000011110000	-3856	0xf0f0
0111011101000010	30530	0x7742
1111111111111010	-6	0xfffa
1111111111111010	-6	0xfffa
1111111111101110	-18	0xffee
0000000000001110	14	0xe
1111000011110000	-3856	0xf0f0
0111011101000011	30531	0x7743
1111111111111110	-2	0xfffe
1111111111111010	-6	0xfffa
1111111111111100	-4	0xfffc
0000000000000100	4	0x4
1111000011110000	-3856	0xf0f0
0111011101000100	30532	0x7744
0000000000000110	6	0x6
1111111111110110	-10	0xfff6
1111111111110010	-14	0xfff2
1111111111111110	-2	0xfffe
1111000011110000	-3856	0xf0f0
0111011101000101	30533	0x7745
1111111111111000	

In [392]:
plt.figure()
plt.plot(np.gradient(values))

In [628]:
values = np.frombuffer(v, dtype=np.int16).copy()[6000:]
for i in range(10):
    if values[i]==-3856:
        values = values[i:].copy()
plt.figure()
for i in range(2,6):
    plt.plot(values[i::6])

In [606]:
plt.figure()
plt.psd(values[3::6].copy(), Fs=2e5, detrend='mean', scale_by_freq=True, NFFT=4096)
#plt.psd(values[3::6].copy(), Fs=2e5, detrend='mean', scale_by_freq=True, NFFT=4096)
#plt.psd(values[4::6].copy(), Fs=2e5, detrend='mean', scale_by_freq=True, NFFT=4096)
#plt.psd(values[5::6].copy(), Fs=2e5, detrend='mean', scale_by_freq=True, NFFT=4096)

(array([4.33121807e-06, 2.37783529e-05, 3.86303005e-05, ...,
        5.80575366e-05, 8.68493859e-05, 3.77967837e-05]),
 array([0.00000000e+00, 4.88281250e+01, 9.76562500e+01, ...,
        9.99023437e+04, 9.99511719e+04, 1.00000000e+05]))

In [589]:
plt.psd?

Signature:
plt.psd(
    x: 'ArrayLike',
    NFFT: 'int | None' = None,
    Fs: 'float | None' = None,
    Fc: 'int | None' = None,
    detrend: "Literal['none', 'mean', 'linear'] | Callable[[ArrayLike], ArrayLike] | None" = None,
    window: 'Callable[[ArrayLike], ArrayLike] | ArrayLike | None' = None,
    noverlap: 'int | None' = None,
    pad_to: 'int | None' = None,
    sides: "Literal['default', 'onesided', 'twosided'] | None" = None,
    scale_by_freq: 'bool | None' = None,
    return_line: 'bool | None' = None,
    *,
    data=None,
    **kwargs,
) -> 'tuple[np.ndarray, np.ndarray] | tuple[np.ndarray, np.ndarray, Line2D]'
Docstring:
Plot the power spectral density.

The power spectral density :math:`P_{xx}` by Welch's average
periodogram method.  The vector *x* is divided into *NFFT* length
segments.  Each segment is detrended by function *detrend* and
windowed by function *window*.  *noverlap* gives the length of
the overlap between segments.  The :math:`|\mathrm{fft}(i)|^2`
of 

In [549]:

for i in range(len(v)):
    if v[i]==0xFF and v[i+1] == 0 and v[i+2]== 0xFF and v[i+3]==0:
        print(i)